In [ ]:
# Required packages (uncomment for one-time installs)
# !pip install ultralytics opencv-python numpy easyocr customtkinter requests pillow

import cv2
import numpy as np
import easyocr
import time
import json
import requests
import os
import threading
import re
from datetime import datetime
from ultralytics import YOLO
from PIL import Image, ImageTk
import customtkinter as ctk
import tkinter as tk

# -------------------------
# LicensePlateDetector (Refactored detection logic)
# -------------------------
class LicensePlateDetector:
    def __init__(self,
                 car_model_path="yolov8n.pt",
                 plate_model_path="LP-detection.pt",
                 superres_model_path="EDSR_x3.pb",
                 use_superres=True,
                 burst_frame_count=5,
                 burst_frame_interval=0.08,
                 cooldown_period=10):

        self.camera_data = {}
        self.cap = None
        self.is_running = False
        self.detection_active = True
        self.last_detection_time = None
        self.cooldown_period = cooldown_period
        self.detection_count = 0

        self.frame_width = 0
        self.frame_height = 0
        self.total_frame_area = 0

        self.consecutive_blank_frames = 0
        self.max_blank_frames = 10

        # Load YOLO models
        print("🚀 Loading YOLO models...")
        self.car_model = YOLO(car_model_path)
        self.plate_model = YOLO(plate_model_path)

        # GPU flag (ultralytics sets device automatically)
        self.use_gpu = (self.car_model.device.type == "cuda")
        print(f"✅ YOLO loaded. Device: {self.car_model.device}")

        # EasyOCR reader
        print("🔤 Initializing EasyOCR reader...")
        try:
            # Use GPU if available
            self.reader = easyocr.Reader(['en'], gpu=self.use_gpu)
            print(f"✅ EasyOCR initialized (gpu={self.use_gpu})")
        except Exception as e:
            print(f"⚠️ EasyOCR init failed, falling back to cpu: {e}")
            self.reader = easyocr.Reader(['en'], gpu=False)

        # Super-resolution
        self.enable_superres = use_superres
        self.superres_model_path = superres_model_path
        self.superres_net = None
        self.initialize_superres()

        # Detection tuning
        # gate_line_y will be set dynamically per-frame as 60% of frame height
        self.gate_line_ratio = 0.60
        self.min_car_width = 120
        self.min_coverage_trigger = 2  # percent

        # Burst capture
        self.burst_frame_count = burst_frame_count
        self.burst_frame_interval = burst_frame_interval

        # Text correction map
        self.subs_map = {
            '0': 'O', 'O': '0', '1': 'I', 'I': '1', '8': 'B', 'B': '8',
            '6': 'G', 'G': '6', '5': 'S', 'S': '5', '2': 'Z', 'Z': '2',
            '4': 'A', 'A': '4', 'U': 'V', 'V': 'U', 'Q': '0'
        }

    def initialize_superres(self):
        if not self.enable_superres:
            return
        try:
            self.superres_net = cv2.dnn_superres.DnnSuperResImpl_create()
            self.superres_net.readModel(self.superres_model_path)
            lower = self.superres_model_path.lower()
            if "edsr" in lower:
                self.superres_net.setModel("edsr", 3)
            elif "fsrcnn" in lower:
                self.superres_net.setModel("fsrcnn", 4)
            elif "lapsrn" in lower or "lapsrn" in lower:
                self.superres_net.setModel("lapsrn", 3)
            else:
                self.superres_net.setModel("edsr", 3)
            print("✅ Super-resolution model loaded.")
        except Exception as e:
            print(f"⚠️ Could not load super-resolution model: {e}")
            self.superres_net = None
            self.enable_superres = False

    def enhance_with_superres(self, image):
        try:
            if self.enable_superres and self.superres_net is not None:
                return self.superres_net.upsample(image)
            return image
        except Exception as e:
            print(f"Error in super-resolution: {e}")
            return image

    # -------------------------
    # Config load (cam.txt)
    # -------------------------
    def load_camera_config(self):
        try:
            with open('cam.txt', 'r') as file:
                lines = [line.strip() for line in file.readlines() if line.strip()]
                # safe access
                self.camera_data = {
                    'rtsp_url': lines[0] if len(lines) > 0 else '',
                    'substream_url': lines[0] if len(lines) > 0 else '',
                    'ip': lines[1].split()[1] if len(lines) > 1 and len(lines[1].split()) > 1 else '192.168.52.160',
                    'port': lines[2].split()[1] if len(lines) > 2 and len(lines[2].split()) > 1 else '554',
                    'id': lines[3].split()[1] if len(lines) > 3 and len(lines[3].split()) > 1 else 'admin',
                    'password': lines[4].split()[1] if len(lines) > 4 and len(lines[4].split()) > 1 else '',
                    'location': lines[5].split()[1] if len(lines) > 5 and len(lines[5].split()) > 1 else 'unknown',
                    'bot_token': lines[6].split()[1] if len(lines) > 6 and len(lines[6].split()) > 1 else '',
                    'chat_id': lines[7].split()[1] if len(lines) > 7 and len(lines[7].split()) > 1 else ''
                }
            print("Camera configuration loaded:", self.camera_data)
            return True
        except Exception as e:
            print(f"Error loading camera config: {e}")
            return False

    # -------------------------
    # Blank frame detection
    # -------------------------
    def check_blank_frame(self, frame):
        if frame is None:
            return True
        if np.mean(frame) < 8:
            return True
        if np.std(frame) < 5:
            return True
        return False

    # -------------------------
    # YOLO-based car detection + NMS
    # -------------------------
    def nms(self, boxes, iou_thresh=0.4):
        """Simple NMS for xyxy boxes: boxes as [x1,y1,x2,y2,conf]"""
        if len(boxes) == 0:
            return []
        boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
        filtered = []
        while boxes:
            chosen = boxes.pop(0)
            filtered.append(chosen)
            keep = []
            for b in boxes:
                # compute iou
                inter_x1 = max(chosen[0], b[0])
                inter_y1 = max(chosen[1], b[1])
                inter_x2 = min(chosen[2], b[2])
                inter_y2 = min(chosen[3], b[3])
                inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
                area_chosen = (chosen[2] - chosen[0]) * (chosen[3] - chosen[1])
                area_b = (b[2] - b[0]) * (b[3] - b[1])
                denom = float(area_chosen + area_b - inter_area) if (area_chosen + area_b - inter_area) > 0 else 1.0
                iou = inter_area / denom
                if iou < iou_thresh:
                    keep.append(b)
            boxes = keep
        return filtered

    def detect_cars_yolo(self, frame, conf_thresh=0.35, iou_thresh=0.45, imgsz=640):
        """Return list of car boxes in [x,y,w,h,area] format"""
        boxes_out = []
        try:
            results = self.car_model.predict(source=frame, conf=conf_thresh, iou=iou_thresh, imgsz=imgsz, verbose=False)
            all_boxes = []
            if results and len(results[0].boxes) > 0:
                for r in results:
                    for box in r.boxes:
                        cls_id = int(box.cls[0].item())
                        conf = float(box.conf[0].item())
                        # COCO class ids: 2 = car, 3 = motorcycle, 5 = bus, 7 = truck
                        if cls_id in [2, 3, 5, 7]:
                            x1, y1, x2, y2 = map(int, box.xyxy[0])
                            area = (x2 - x1) * (y2 - y1)
                            all_boxes.append([x1, y1, x2, y2, conf, area])
            # convert to [x1,y1,x2,y2,conf] for nms
            nms_input = [[b[0], b[1], b[2], b[3], b[4]] for b in all_boxes]
            nmsed = self.nms(nms_input, iou_thresh=0.4)
            # convert to desired output format and filter by min area
            for x1, y1, x2, y2, conf in nmsed:
                w = x2 - x1
                h = y2 - y1
                area = w * h
                boxes_out.append((x1, y1, w, h, area))
        except Exception as e:
            print(f"Error in detect_cars_yolo: {e}")
        return boxes_out

    # -------------------------
    # Plate detection with YOLO + merge
    # -------------------------
    def merge_boxes(self, boxes, iou_thresh=0.4):
        """Merge overlapping YOLO boxes that likely belong to the same plate.
           boxes: list of [x1,y1,x2,y2,conf]"""
        if not boxes:
            return []
        boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
        merged = []
        while boxes:
            base = boxes.pop(0)
            x1b, y1b, x2b, y2b, confb = base
            keep = [base]
            remove_idx = []
            for i, box in enumerate(boxes):
                x1, y1, x2, y2, conf = box
                inter_x1 = max(x1, x1b)
                inter_y1 = max(y1, y1b)
                inter_x2 = min(x2, x2b)
                inter_y2 = min(y2, y2b)
                inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
                area1 = (x2 - x1) * (y2 - y1)
                area2 = (x2b - x1b) * (y2b - y1b)
                denom = (area1 + area2 - inter_area) if (area1 + area2 - inter_area) > 0 else 1.0
                iou = inter_area / denom
                if iou > iou_thresh:
                    keep.append(box)
                    remove_idx.append(i)
            for idx in sorted(remove_idx, reverse=True):
                boxes.pop(idx)
            merged_box = [
                min(b[0] for b in keep),
                min(b[1] for b in keep),
                max(b[2] for b in keep),
                max(b[3] for b in keep),
                max(b[4] for b in keep)
            ]
            merged.append(merged_box)
        return merged

    def detect_plates_yolo(self, frame, car_box=None, conf_thresh=0.18, iou_thresh=0.45, imgsz=960):
        """
        Run plate model either on whole frame or ROI (car_box).
        Returns best plate box (x,y,w,h) or None.
        """
        try:
            if car_box:
                x, y, w, h, area = car_box
                pad_w = int(0.12 * w)
                pad_h = int(0.12 * h)
                x1 = max(0, x - pad_w)
                y1 = max(0, y - pad_h)
                x2 = min(frame.shape[1], x + w + pad_w)
                y2 = min(frame.shape[0], y + h + pad_h)
                roi = frame[y1:y2, x1:x2]
                if roi is None or roi.size == 0:
                    return None
                results = self.plate_model.predict(source=roi, conf=conf_thresh, iou=iou_thresh, imgsz=imgsz, verbose=False)
                plate_boxes = []
                for r in results:
                    for box in r.boxes:
                        xA, yA, xB, yB = map(int, box.xyxy[0])
                        conf = float(box.conf[0].item())
                        abs_x1 = x1 + xA
                        abs_y1 = y1 + yA
                        abs_x2 = x1 + xB
                        abs_y2 = y1 + yB
                        plate_boxes.append([abs_x1, abs_y1, abs_x2, abs_y2, conf])
            else:
                results = self.plate_model.predict(source=frame, conf=conf_thresh, iou=iou_thresh, imgsz=imgsz, verbose=False)
                plate_boxes = []
                for r in results:
                    for box in r.boxes:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        conf = float(box.conf[0].item())
                        plate_boxes.append([x1, y1, x2, y2, conf])

            if not plate_boxes:
                return None

            merged = self.merge_boxes(plate_boxes, iou_thresh=0.35)

            best = None
            for (x1m, y1m, x2m, y2m, confm) in merged:
                w_m = x2m - x1m
                h_m = y2m - y1m
                if h_m <= 0:
                    continue
                ar = w_m / float(h_m)
                if 1.0 <= ar <= 7.0 and w_m > 30 and h_m > 10:
                    best = (x1m, y1m, w_m, h_m)
                    break

            if best is None:
                merged_sorted = sorted(merged, key=lambda b: b[4], reverse=True)
                if merged_sorted:
                    x1m, y1m, x2m, y2m, confm = merged_sorted[0]
                    best = (x1m, y1m, x2m - x1m, y2m - y1m)

            return best
        except Exception as e:
            print(f"Error in detect_plates_yolo: {e}")
            return None

    # -------------------------
    # Preprocess plate image and OCR using EasyOCR (with multiple attempts)
    # -------------------------
    def preprocess_plate_image(self, plate_image):
        try:
            if plate_image is None or plate_image.size == 0:
                return []

            # Try super-resolution first if enabled
            try:
                if self.enable_superres and self.superres_net is not None:
                    plate_image = self.superres_net.upsample(plate_image)
            except Exception as e:
                print(f"Super-res skipped: {e}")

            # Ensure color -> gray
            gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)

            # Resize (scale up) to improve OCR
            gray = cv2.resize(gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)

            # Denoise + sharpen + contrast
            gray = cv2.GaussianBlur(gray, (3, 3), 0)
            gray = cv2.equalizeHist(gray)
            gray = cv2.bilateralFilter(gray, 9, 75, 75)
            gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=15)
            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
            gray = cv2.filter2D(gray, -1, kernel)

            # Binary versions
            _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                             cv2.THRESH_BINARY, 11, 2)

            return [gray, thresh, adaptive]
        except Exception as e:
            print(f"Error in image preprocessing: {e}")
            return []

    def correct_text(self, text):
        text = re.sub(r'[^A-Z0-9]', '', text.upper())
        corrected = ''.join([self.subs_map.get(ch, ch) for ch in text])
        # conservative pattern for Indian plates - lenient in case of partial reads
        pattern = r'[A-Z]{2}\d{1,2}[A-Z]{0,3}\d{1,4}'
        if not re.fullmatch(pattern, corrected):
            # try replacements again
            for k, v in self.subs_map.items():
                corrected = corrected.replace(k, v)
        return corrected

    def extract_text_from_plate(self, plate_image):
        try:
            if plate_image is None or plate_image.size == 0:
                return None
            preprocessed_imgs = self.preprocess_plate_image(plate_image)
            if not preprocessed_imgs:
                return None

            text_found = None
            # Try multiple preprocessed images
            for ocr_img in preprocessed_imgs:
                try:
                    ocr_result = self.reader.readtext(ocr_img, detail=0, paragraph=False)
                except Exception as e:
                    try:
                        ocr_result = self.reader.readtext(cv2.cvtColor(ocr_img, cv2.COLOR_GRAY2BGR), detail=0, paragraph=False)
                    except Exception:
                        ocr_result = []
                if ocr_result:
                    text_found = max(ocr_result, key=len)
                    break

            if not text_found:
                return None

            text_clean = self.correct_text(text_found)
            if 2 <= len(text_clean) <= 15:
                return text_clean
            return None
        except Exception as e:
            print(f"Error in text extraction: {e}")
            return None

    # -------------------------
    # Save images, send telegram
    # -------------------------
    def save_detection_images(self, frame, car_region, plate_region, plate_text, coverage_percentage):
        try:
            annotated_frame = frame.copy()
            car_x, car_y, car_w, car_h, car_area = car_region
            cv2.rectangle(annotated_frame, (car_x, car_y), (car_x + car_w, car_y + car_h), (0, 255, 255), 2)

            if plate_region:
                px, py, pw, ph = plate_region
                cv2.rectangle(annotated_frame, (px, py), (px + pw, py + ph), (0, 255, 0), 2)

            cv2.putText(annotated_frame, f"Coverage: {coverage_percentage}%", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(annotated_frame, f"Plate: {plate_text}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(annotated_frame, f"Location: {self.camera_data.get('location','-')}", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(annotated_frame, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            full_filename = f"full_detection_{self.detection_count}.jpg"
            closeup_filename = f"closeup_{self.detection_count}.jpg"

            cv2.imwrite(full_filename, annotated_frame)

            if plate_region:
                px, py, pw, ph = plate_region
                plate_closeup = frame[py:py + ph, px:px + pw]
                if plate_closeup.size > 0:
                    closeup_enhanced = cv2.resize(plate_closeup, (int(pw * 3), int(ph * 3)), interpolation=cv2.INTER_CUBIC)
                    cv2.imwrite(closeup_filename, closeup_enhanced)
                else:
                    car_closeup = frame[car_y:car_y + car_h, car_x:car_x + car_w]
                    car_closeup = cv2.resize(car_closeup, (300, 200), interpolation=cv2.INTER_CUBIC)
                    cv2.putText(car_closeup, "Plate Not Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.imwrite(closeup_filename, car_closeup)
            else:
                car_closeup = frame[car_y:car_y + car_h, car_x:car_x + car_w]
                car_closeup = cv2.resize(car_closeup, (300, 200), interpolation=cv2.INTER_CUBIC)
                cv2.putText(car_closeup, "Plate Not Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.imwrite(closeup_filename, car_closeup)

            return full_filename, closeup_filename
        except Exception as e:
            print(f"Error saving images: {e}")
            return None, None

    def send_to_telegram(self, full_image_path, closeup_image_path, plate_text, coverage_percentage):
        try:
            bot = self.camera_data.get('bot_token')
            chat = self.camera_data.get('chat_id')
            if not bot or not chat:
                print("Telegram bot token or chat id missing.")
                return False

            url = f"https://api.telegram.org/bot{bot}/sendMediaGroup"
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            caption = f"🚙 Detection #{self.detection_count}\n"
            caption += f"📍 {self.camera_data.get('location','-')}\n"
            caption += f"📅 {current_time}\n"
            caption += f"📏 Coverage: {coverage_percentage}%\n"
            caption += f"🔢 Plate: `{plate_text}`"

            media = [
                {'type': 'photo', 'media': 'attach://full_image.jpg', 'caption': caption, 'parse_mode': 'Markdown'},
                {'type': 'photo', 'media': 'attach://closeup_image.jpg'}
            ]
            files = {
                'full_image.jpg': open(full_image_path, 'rb'),
                'closeup_image.jpg': open(closeup_image_path, 'rb')
            }
            data = {'chat_id': chat, 'media': json.dumps(media)}
            response = requests.post(url, files=files, data=data)
            files['full_image.jpg'].close()
            files['closeup_image.jpg'].close()
            if response.status_code == 200:
                print(f"✅ Alert sent: {plate_text}")
                return True
            else:
                print(f"❌ Telegram error: {response.text}")
                return False
        except Exception as e:
            print(f"❌ Error sending to Telegram: {e}")
            return False

    # -------------------------
    # Burst capture
    # -------------------------
    def capture_burst_frames(self, cap, count=5, interval=0.08):
        frames = []
        try:
            for _ in range(count):
                ret, f = cap.read()
                if not ret:
                    break
                frames.append(f.copy())
                time.sleep(interval)
            return frames
        except Exception as e:
            print(f"⚠️ Burst capture failed: {e}")
            return frames

    # -------------------------
    # Main per-frame processing (YOLO core + gate-line trigger)
    # -------------------------
    def process_frame(self, frame):
        if not self.detection_active:
            return frame, None, None, None, 0

        current_time = datetime.now()
        if self.last_detection_time and (current_time - self.last_detection_time).seconds < self.cooldown_period:
            return frame, None, None, None, 0

        try:
            self.frame_height, self.frame_width = frame.shape[:2]
            self.total_frame_area = self.frame_width * self.frame_height

            # dynamic gate line (60% of frame height)
            self.gate_line_y = int(self.frame_height * self.gate_line_ratio)

            if self.check_blank_frame(frame):
                self.consecutive_blank_frames += 1
                if self.consecutive_blank_frames >= self.max_blank_frames:
                    raise Exception("Camera stream stuck/blank")
                return frame, None, None, None, 0
            else:
                self.consecutive_blank_frames = 0

            # 1) detect cars using YOLO + NMS
            car_boxes = self.detect_cars_yolo(frame)
            # annotate for debug gate-line
            cv2.line(frame, (0, self.gate_line_y), (self.frame_width, self.gate_line_y), (255, 0, 0), 2)

            for car_box in car_boxes:
                x, y, w, h, area = car_box
                coverage_percentage = round((area / float(self.total_frame_area)) * 100, 2) if self.total_frame_area > 0 else 0
                car_center_y = y + h // 2

                # Gate-line trigger or large car or coverage threshold
                if (car_center_y >= self.gate_line_y or w >= self.min_car_width or coverage_percentage >= self.min_coverage_trigger):
                    print(f"🚗 Triggered: coverage={coverage_percentage}%")
                    self.last_detection_time = current_time

                    # 2) detect plate inside car ROI
                    plate_region = self.detect_plates_yolo(frame, (x, y, w, h, area))
                    plate_text = None
                    if plate_region:
                        px, py, pw, ph = plate_region
                        plate_image = frame[py:py + ph, px:px + pw]
                        plate_text = self.extract_text_from_plate(plate_image)
                        if plate_text:
                            print(f"🔢 Plate read: {plate_text}")

                    self.detection_count += 1
                    self.last_detection_time = current_time

                    # burst capture (choose best sharp frame)
                    if hasattr(self, "cap") and self.cap is not None and self.cap.isOpened():
                        burst_frames = self.capture_burst_frames(self.cap, self.burst_frame_count, self.burst_frame_interval)
                    else:
                        burst_frames = [frame.copy()]

                    best_frame = max(burst_frames, key=lambda f: cv2.Laplacian(cv2.cvtColor(f, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()) if burst_frames else frame.copy()
                    frame = best_frame.copy()

                    # spawn worker thread to save/send
                    threading.Thread(
                        target=self.handle_detection,
                        args=(frame.copy(), (x, y, w, h, area), plate_region, plate_text, coverage_percentage),
                        daemon=True
                    ).start()

                    return frame, (x, y, w, h, area), plate_region, plate_text, coverage_percentage
                else:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(frame, f"Too far ({int(coverage_percentage)}%)", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        except Exception as e:
            print(f"Error processing frame: {e}")
            raise e

        return frame, None, None, None, 0

    def handle_detection(self, frame, car_region, plate_region, plate_text, coverage_percentage):
        try:
            print(f"🎯 Detected (coverage {coverage_percentage}%) | Plate: {plate_text or 'NOT DETECTED'}")
            full_image_path, closeup_image_path = self.save_detection_images(frame, car_region, plate_region, plate_text or "NOT DETECTED", coverage_percentage)
            if full_image_path and closeup_image_path:
                sent = self.send_to_telegram(full_image_path, closeup_image_path, plate_text or "NOT DETECTED", coverage_percentage)
                if sent:
                    print(f"✅ Detection #{self.detection_count} processed.")
                else:
                    print(f"❌ Detection #{self.detection_count} send failed.")
                # cleanup
                try:
                    os.remove(full_image_path)
                    os.remove(closeup_image_path)
                except Exception as e:
                    print(f"Warning: could not delete temp images: {e}")
        except Exception as e:
            print(f"❌ Error handling detection: {e}")

# -------------------------
# GUI (LicensePlateGUI) - unchanged visually, uses new detector underneath
# -------------------------
class LicensePlateGUI:
    def __init__(self, root):
        ctk.set_appearance_mode("Dark")
        ctk.set_default_color_theme("blue")

        self.root = root
        self.root.title("🚗 Car Detection Dashboard")
        self.root.geometry("720x480")
        self.root.resizable(False, False)
        self.root.attributes('-topmost', True)

        # Detector backend
        self.detector = LicensePlateDetector()

        self.car_count = 0
        self.setup_gui()

    def setup_gui(self):
        self.main_frame = ctk.CTkFrame(self.root, corner_radius=10)
        self.main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        header_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        header_frame.pack(fill="x", pady=(5, 0))
        self.title_label = ctk.CTkLabel(header_frame, text="Car Detection Live Stream", font=ctk.CTkFont(size=18, weight="bold"))
        self.title_label.pack(side="left", padx=10)
        self.theme_switch = ctk.CTkSwitch(header_frame, text="Light Mode", command=self.toggle_theme)
        self.theme_switch.pack(side="right", padx=10)

        video_frame = ctk.CTkFrame(self.main_frame, corner_radius=10)
        video_frame.pack(fill="both", expand=True, pady=10, padx=10)

        self.counter_label = ctk.CTkLabel(video_frame, text="Cars: 0", font=ctk.CTkFont(size=13, weight="bold"),
                                          fg_color="#1E88E5", text_color="white", corner_radius=8, padx=10, pady=3)
        self.counter_label.place(x=10, y=10)

        self.video_label = ctk.CTkLabel(video_frame, text="Camera feed offline.\nPress Start to begin.",
                                        font=ctk.CTkFont(size=13), height=280, corner_radius=8,
                                        fg_color=("gray90", "#2b2b2b"), justify="center")
        self.video_label.pack(fill="both", expand=True, padx=10, pady=10)

        controls_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        controls_frame.pack(fill="x", pady=(0, 5))

        self.start_btn = ctk.CTkButton(controls_frame, text="Start", width=80, command=self.start_detection)
        self.start_btn.pack(side="left", padx=5)
        self.pause_btn = ctk.CTkButton(controls_frame, text="Pause", width=80, command=self.toggle_detection)
        self.pause_btn.pack(side="left", padx=5)
        self.stop_btn = ctk.CTkButton(controls_frame, text="Stop", width=80, command=self.stop_detection, state="disabled")
        self.stop_btn.pack(side="left", padx=5)

        bottom_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        bottom_frame.pack(fill="x", pady=(5, 0))

        self.status_label = ctk.CTkLabel(bottom_frame, text="Status: Idle", text_color="orange", font=ctk.CTkFont(size=12, weight="bold"))
        self.status_label.pack(side="left", padx=10)
        self.log_label = ctk.CTkLabel(bottom_frame, text="Ready - Integrated YOLO + EasyOCR", font=ctk.CTkFont(size=11), text_color="gray")
        self.log_label.pack(side="right", padx=10)

    def toggle_theme(self):
        if self.theme_switch.get():
            ctk.set_appearance_mode("Light")
            self.theme_switch.configure(text="Dark Mode")
        else:
            ctk.set_appearance_mode("Dark")
            self.theme_switch.configure(text="Light Mode")

    def log_message(self, message):
        short_msg = message[:60] + "..." if len(message) > 60 else message
        self.log_label.configure(text=short_msg)
        self.root.update()

    def start_detection(self):
        try:
            if not self.detector.load_camera_config():
                self.log_message("❌ Config load failed")
                return

            self.log_message("📋 Loading camera...")

            substream = self.detector.camera_data.get('substream_url') or self.detector.camera_data.get('rtsp_url')
            self.detector.cap = cv2.VideoCapture(substream)
            self.detector.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
            self.detector.cap.set(cv2.CAP_PROP_FPS, 18)
            self.detector.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.detector.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)

            if not self.detector.cap.isOpened():
                self.log_message("❌ Camera connect failed")
                return

            self.detector.is_running = True
            self.start_btn.configure(state="disabled")
            self.stop_btn.configure(state="normal")
            self.status_label.configure(text="Status: Running", text_color="green")
            self.log_message("✅ System started - Hybrid YOLO + EasyOCR (gate-line preserved)")

            # attach cap to detector for burst capture
            self.detector.cap = self.detector.cap
            self.process_video()
        except Exception as e:
            self.log_message(f"❌ Start error: {str(e)[:60]}")

    def stop_detection(self):
        self.detector.is_running = False
        if self.detector.cap:
            try:
                self.detector.cap.release()
            except Exception:
                pass
            self.detector.cap = None

        self.start_btn.configure(state="normal")
        self.stop_btn.configure(state="disabled")
        self.status_label.configure(text="Status: Stopped", text_color="red")
        self.video_label.configure(image='', text="Camera feed offline.\nPress Start to begin.")
        self.log_message("⏹ System stopped")

    def toggle_detection(self):
        self.detector.detection_active = not self.detector.detection_active
        status = "ACTIVE" if self.detector.detection_active else "PAUSED"
        color = "green" if self.detector.detection_active else "orange"
        self.pause_btn.configure(text="Resume" if not self.detector.detection_active else "Pause")
        self.status_label.configure(text=f"Status: {status}", text_color=color)
        self.log_message(f"⏸ Detection {status.lower()}")

    def process_video(self):
        if not self.detector.is_running or not self.detector.cap:
            return

        try:
            ret, frame = self.detector.cap.read()
            if ret:
                processed_frame, car_region, plate_region, plate_text, coverage = self.detector.process_frame(frame)

                if car_region:
                    self.car_count += 1
                    self.counter_label.configure(text=f"Cars: {self.car_count}")

                rgb_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(rgb_frame)
                img = img.resize((640, 360), Image.Resampling.LANCZOS)
                img_tk = ImageTk.PhotoImage(image=img)

                self.video_label.configure(image=img_tk, text="")
                self.video_label.image = img_tk
            else:
                self.log_message("⚠️ Frame read failed - reconnecting")
                self.root.after(1000, self.reconnect_camera)
                return

        except Exception as e:
            self.log_message(f"❌ Video error: {str(e)[:60]}")
            self.root.after(1000, self.reconnect_camera)
            return

        if self.detector.is_running:
            self.root.after(55, self.process_video)

    def reconnect_camera(self):
        self.log_message("🔄 Reconnecting...")
        self.stop_detection()
        self.root.after(2000, self.start_detection)

# -------------------------
# main
# -------------------------
def main():
    try:
        root = ctk.CTk()
        app = LicensePlateGUI(root)

        window_width = 720
        window_height = 480
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        x_position = screen_width - window_width - 10
        y_position = 10
        root.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")

        root.mainloop()
    except Exception as e:
        print(f"Application error: {e}")

if __name__ == "__main__":
    main()
# done 